In [197]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import random as rm
import re

In [198]:
#Empezamos la exploración de los datos cargando los datos en dataframes
actors = pd.read_csv('../ignored/data_raw/actor.csv',encoding='utf-8',encoding_errors= 'ignore')
categories = pd.read_csv('../ignored/data_raw/category.csv',encoding='utf-8',encoding_errors= 'ignore')
films = pd.read_csv('../ignored/data_raw/film.csv',encoding='utf-8',encoding_errors= 'ignore')
inventory = pd.read_csv('../ignored/data_raw/inventory.csv',encoding='utf-8',encoding_errors= 'ignore')
languages = pd.read_csv('../ignored/data_raw/language.csv',encoding='utf-8',encoding_errors= 'ignore')
old = pd.read_csv('../ignored/data_raw/old_HDD.csv',encoding='utf-8',encoding_errors= 'ignore')
rent = pd.read_csv('../ignored/data_raw/rental.csv',encoding='utf-8',encoding_errors= 'ignore')

In [199]:
films.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [200]:
#Tras una observación exshaustiva de los datos me doy cuenta de que necesito tener una columna de Category_ID en el df de films que más adelante
#rellenaré con las propias categorías. Como no están todas las películas en old, algunas se quedarán en nan. Este sería el momento de tirarle 
#de las orejas a quien te ha mandado esto porque te FALTAN DATOS 


films['category_id'] = None

for e in range(len(old['title'])):
    for i in range(len(films['title'])):
        if old['title'][e] == films['title'][i]:
            films['category_id'][i] = old['category_id'][e]




C:\Users\pacot\AppData\Local\Temp\ipykernel_10032\539342624.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  films['category_id'][i] = old['category_id'][e]


In [201]:
#además elimino la columna de original_language_id ya que veo que a penas proporcionaría información siendo que ya tengo una relación 
#establecida entre mi película y su lenguage. También traslado la columna de lenguage al inventario ya que considero mucho más útil 
#tener en películas valores únicos de título y, ya en el inventario donde "repito" películas, introducir en que lenguage va cada registro

inventory['language_id'] = 1 #según los datos que tenemos solo poseemos películas en inglés con lo cual me vale con rellenar de 1
films.drop(['original_language_id','language_id'],axis = 1,inplace=True)

In [202]:
#ahora traduzcamos la tabla old para que nos haga de conexión entre actores y títulos en una relación many to many
actors['actor'] = actors['first_name'] + ' ' + actors['last_name']
old['actor'] = old['first_name'] + ' ' + old['last_name']


In [203]:
#Rellenamos actor_id
old['actor_id'] = None

for e in range(len(old['actor'])):
    for i in range(len(actors['actor'])):
        if old['actor'][e] == actors['actor'][i]:
            old['actor_id'][e] = actors['actor_id'][i]


C:\Users\pacot\AppData\Local\Temp\ipykernel_10032\1716096900.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old['actor_id'][e] = actors['actor_id'][i]


In [204]:
#Rellenamos film_id
old['film_id'] = None

for e in range(len(old['title'])):
    for i in range(len(films['title'])):
        if old['title'][e] == films['title'][i]:
            old['film_id'][e] = films['film_id'][i]

C:\Users\pacot\AppData\Local\Temp\ipykernel_10032\3822256434.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old['film_id'][e] = films['film_id'][i]


In [205]:
#me deshago de las columnas sobrantes en old y en actor
old.drop(['first_name','last_name','title','release_year','category_id','actor'],axis = 1,inplace=True)
actors.drop(['actor'],axis = 1,inplace=True)


In [206]:
#solo tengo una tienda de momento con lo que todos los store_id a 1
inventory['store_id'] = 1
inventory.head()

,inventory_id,film_id,store_id,last_update,language_id
0,1,1,1,2006-02-15 05:09:17,1
1,2,1,1,2006-02-15 05:09:17,1
2,3,1,1,2006-02-15 05:09:17,1
3,4,1,1,2006-02-15 05:09:17,1
4,5,1,1,2006-02-15 05:09:17,1


In [216]:
#renombro la columna referente al precio en la película
films.columns = ['film_id','title','description','release_year','rental_duration','price','length','replacement_cost','rating','special_features','last_update','category_id']
films.head()

,film_id,title,description,release_year,rental_duration,price,length,replacement_cost,rating,special_features,last_update,category_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42,11
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42,6
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42,None
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42,None


In [220]:
#modifico el orden para dejarlo igual al de sql
inventory = inventory.reindex(columns=['inventory_id','film_id','store_id','language_id','last_update'])
inventory.head()

,inventory_id,film_id,store_id,language_id,last_update
0,1,1,1,1,2006-02-15 05:09:17
1,2,1,1,1,2006-02-15 05:09:17
2,3,1,1,1,2006-02-15 05:09:17
3,4,1,1,1,2006-02-15 05:09:17
4,5,1,1,1,2006-02-15 05:09:17


In [208]:
for e in films['special_features']:
    print (len(e))

32
23
23
30
14
14
23
8
23
23
30
27
32
41
39
21
26
26
45
45
30
30
23
26
8
27
32
32
36
41
45
21
54
27
41
21
23
54
8
21
41
26
17
26
26
54
12
26
39
30
14
27
45
36
17
21
14
32
14
32
21
36
41
30
41
27
14
45
8
27
45
21
21
36
30
45
26
23
26
8
26
39
41
21
14
30
17
36
45
8
17
23
8
23
36
21
39
8
8
12
17
54
27
12
14
8
17
36
17
27
27
39
36
54
17
32
23
23
36
27
26
39
41
8
30
8
27
26
45
32
14
32
8
8
45
17
41
32
21
39
32
54
36
36
39
12
30
39
39
17
14
41
27
27
41
12
54
17
26
23
39
41
45
54
23
30
12
26
30
17
36
32
17
12
54
30
27
32
45
21
12
12
12
14
45
32
12
27
23
26
45
45
39
39
26
32
12
21
17
54
23
32
39
36
54
30
12
30
32
54
30
14
12
8
41
54
27
54
30
14
14
26
14
14
39
39
27
39
26
30
26
45
41
26
21
21
36
39
12
27
39
21
54
12
41
32
36
45
12
32
26
39
26
41
23
21
45
8
17
23
12
17
21
27
17
14
39
39
17
36
39
14
12
30
45
30
17
39
41
14
26
8
30
8
14
12
45
8
36
21
17
12
45
14
26
8
36
21
36
30
8
12
17
14
26
23
32
41
54
27
54
32
41
54
21
8
39
27
41
8
39
36
8
54
39
39
21
45
12
26
39
32
26
39
39
17
23
45
41
27
23
3